# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Import what you need

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [2]:
driver = webdriver.Chrome()
url = "https://jportal.mdcourts.gov/license/pbPublicSearch.jsp"

In [3]:
driver.get(url)

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages, or you can close and re-open Chrome.

- Check the checkbox, then submit the form to accept their terms of service

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

- *TIP: if something doesn't have anything special about it, xpath might be your best bet*

In [4]:
driver.get(url)
disclaimer_button = driver.find_element_by_xpath('//*[@id="checkbox"]')
disclaimer_button.click()

enter_button = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
enter_button.click()

Now click the **Search License Records** link up top in the navigation to get to the search page.

- *TIP: Honestly you could also just visit the URL directly now since you filled out that terms of service thing*

In [5]:
driver.get(url)

In [ ]:
search_button = driver.find_element_by_xpath('/html/body/table[1]/tbody/tr[2]/td[2]/table/tbody/tr/td[3]/a/img')
search_button.click()

### Perform your search

Pick "Statewide" for the jurisdiction dropdown, and `VAP%` into the Trade Name field. The `%` is a wildcard.

In [6]:
statewide_dropdown = Select(driver.find_element_by_name('slcJurisdiction'))
statewide_dropdown.select_by_visible_text('Statewide')

In [7]:
vape_shops = driver.find_element_by_id("txtTradeName")
vape_shops.send_keys("VAP%") 

In [8]:
submit_search_button = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]')
submit_search_button.click()

And now, of course, **submit the form**.

- *TIP: Since scrolling to buttons can be a pain, sometimes it's easier to select the form and use `.submit()` instead of `.click()`ing the button*

## (Try to) scrape the results

Let's start by just **printing this stuff**. We'll save it as a dataframe later on.

For now, just scrape **each store's name**, then cry a little. Fact: this is an impossible and miserable page. 

In [9]:
shop_names = driver.find_elements_by_class_name("searchfieldtitle")
for name in shop_names:
    print(name.text)

1. VAPE IT STORE II
2. VAPE IT STORE I
3. VAPEPAD THE
4. VAPE FROG Pending *
5. VAPE FROG Pending *


To avoid struggling with the search results page, we're going to use the **detail page** instead. Try to figure out how to select it and click it inside of your `for` loop.

- *TIP: Instead of just looking for an `a` or an `img`, you might want to look for one of its parents first, then click. This might affect the way you print out the shop's name, too*
- *TIP: Not all of them have links! You can wrap in try/except to skip it, or you can check to see if the shop's status is Pending.*

In [10]:
shop_names = driver.find_elements_by_tag_name("table")[3].find_elements_by_class_name("searchfieldtitle")

In [11]:
shop_names[4].text

'5. VAPE FROG Pending *'

In [12]:
shop_names = driver.find_elements_by_class_name("searchfieldtitle")
for name in shop_names:
    try:
        shop_names_url = name.find_elements_by_tag_name("td")[2].find_element_by_tag_name("a").get_attribute('href')
        print(shop_names_url)
#        driver.get(shop_names_url)

    except:
        if 'Pending' in name.text:
            print("Status pending")

https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=pPfKAhNUVco%3D
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=KVyHlaQ1Wxs%3D
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=xCkmCTJY5kU%3D
Status pending
Status pending


Okay, now let's get to action. For each result, **click the link to the detail page** and print out the following information:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

If it doesn't have a detail page, just print out the name and that's all we need.

- *TIP: When you're done getting the information, you probably want to click back to the search results*
- *TIP: You might enjoy `find_element_by_partial_link_text` to do that*
- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*

> **IMPORTANT NOTE:** This is doomed. It's useful to do, but your current process is doomed. Once you get a `stale element reference` error move on to the next cell.

In [13]:
shop_names = driver.find_elements_by_class_name("searchfieldtitle")
for name in shop_names:
    try:
        shop_names_url = name.find_elements_by_tag_name("td")[2].find_element_by_tag_name("a").get_attribute('href')
        driver.get(shop_names_url)
        
        addresses = driver.find_elements_by_tag_name("table")[3]
        mailing_add = addresses.find_elements_by_tag_name('tr')[2].find_elements_by_tag_name('td')[0].text
        mailing_add_clean = mailing_add.replace("\n", "")
        mailing_add_clean
        
        location_add = addresses.find_elements_by_tag_name('tr')[4].find_elements_by_tag_name('td')[0].text
        location_add_clean = location_add.replace("\n", "")
        location_add_clean
        
        license_info = addresses.find_elements_by_tag_name('tr')[2].find_elements_by_tag_name('td')[1].text
        license_info_clean = license_info.replace("\n", "")
        license_info_clean
        
        amount_paid = driver.find_elements_by_tag_name("table")[4]
        total_amount = amount_paid.find_elements_by_tag_name('tr')[7].find_elements_by_tag_name('td')[1].text
        total_amount
        
        issued = driver.find_elements_by_tag_name("table")[5]
        issued_by = issued.find_elements_by_tag_name('tr')[1].text
        issued_by_clean = issued_by.replace("\n", "")
        issued_by_clean
        
        
        print(mailing_add_clean, location_add_clean, license_info_clean, total_amount, issued_by_clean)
        
        return_link = driver.find_element_by_partial_link_text("Return to")
        return_link.click()

    except:
        if 'Pending' in name.text:
            print("Status pending")

AMIN NARGIS1104 PLANTERS PLACESALISBURY, MD 21804 VAPE IT STORE II1015 S SALISBURY BLVDSALISBURY, MD 21801 License Status: IssuedLicense No.: 22375606Control No.: 22884439Date of Issue: 4/27/2018Months Paid: 12Exp. Date: 4/30/2019Subdivision: 22 Salisbury $ 41.00 WICOMICO COUNTY, CLERK OF CIRCUIT COURTP.O. BOX 198SALISBURY, MARYLAND 21803-0198


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.3 x86_64)


### Stale message reference

Once you navigate away from a page, and you go back to it, you can't use the variables from the first time you were on the page. So, we got a list of results when we first visited, clicked to the details page, clicked back, and now our original list is "stale."

This is sad.

Let's try this again: loop through the results and create a dataframe with `name` and `url` columns. And yes, some of them won't have URLs.

In [14]:
shops = []

shop_names = driver.find_elements_by_class_name("searchfieldtitle")
for name in shop_names:
    shop_dict = {}
    
    shop_dict["name"] = name.text
    if 'Pending' in name.text:
        shop_dict['name'] = name.text.replace(" Pending *", "")

    try:
        shop_names_url = name.find_elements_by_tag_name("td")[2].find_element_by_tag_name("a").get_attribute('href')
        shop_dict["url"] = shop_names_url
        
    except:
            shop_dict["url"] = "Pending"
    
    shops.append(shop_dict)

df = pd.DataFrame(shops)
df


,name,url
0,1. VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,2. VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,3. VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,4. VAPE FROG,Pending
4,5. VAPE FROG,Pending


### Getting all of the results

After you've looped through the results on one page, we're going to want to go to the next page! Add a line to make it click the 'Next' button down at the bottom

- *Tip: `find_element_by_partial_link_text` will be your friend*
- *Tip: You might need to do the scrolling thing to get it onto the screen (and by that I mean, you WILL need to, so you should)*

Confirm that it moves to the next page (it doesn't need to scrape anything yet)

In [15]:
shops = []

shop_names = driver.find_elements_by_class_name("searchfieldtitle")
for name in shop_names:
    shop_dict = {}
    
    shop_dict["name"] = name.text
    if 'Pending' in name.text:
        shop_dict['name'] = name.text.replace(" Pending *", "")

    try:
        shop_names_url = name.find_elements_by_tag_name("td")[2].find_element_by_tag_name("a").get_attribute('href')
        shop_dict["url"] = shop_names_url
        
    except:
            shop_dict["url"] = "Pending"
    
    shops.append(shop_dict)
    
    next_button = driver.find_element_by_partial_link_text('Next')
    next_button.click()

    


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.3 x86_64)


### Wrapping with `while`

> Go back to the first page of results before you try to run this

You have a bunch of scraping code. It clicks the next button, then it stops. But you'd like it to go back up to the top! You can make that happen with a special `while` loop.

```python
while True:
    # Scrape your stuff
    # Click next button
```

This will go on FOREVER AND EVER until there is an error (when it can't find the Next button on the last page of results, you'll get an error).

- *Tip: Print out "Scraping a new page" every time you visit a new page, just to check that it's working*

In [17]:
while True:
    shops = []
    shop_names = driver.find_elements_by_class_name("searchfieldtitle")

    for name in shop_names:
        shop_dict = {}
        
        shop_dict["name"] = name.text
        if 'Pending' in name.text:
            shop_dict['name'] = name.text.replace(" Pending *", "")
        
        try:
            shop_names_url = name.find_elements_by_tag_name("td")[2].find_element_by_tag_name("a").get_attribute('href')
            shop_dict["url"] = shop_names_url
            
        except:
                shop_dict["url"] = "Pending"
        
        shops.append(shop_dict)
        
    next_button = driver.find_element_by_partial_link_text('Next')
    next_button.click()
    print("Scraping a new page")

Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page
Scraping a new page


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Next"}
  (Session info: chrome=67.0.3396.87)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.3 x86_64)


### Making it perfect

> Go back to the first page of results before you try to run this

Wrap all of your code in a `try`/`except` so that it doesn't finish with an error and you'll be good to go.

**Confirm your list has all of the vape shops in it.** If not, check where you are creating your empty list (`[]`) - if you do it in the wrong spot, it will overwrite your list every time you visit a page.

In [18]:
shops = []

try:
    
    while True:
        shop_names = driver.find_elements_by_class_name("searchfieldtitle")
        for name in shop_names:
            shop_dict = {}
            
            shop_dict["name"] = name.text
            if 'Pending' in name.text:
                shop_dict['name'] = name.text.replace(" Pending *", "")
            
            try:
                shop_names_url = name.find_elements_by_tag_name("td")[2].find_element_by_tag_name("a").get_attribute('href')
                shop_dict["url"] = shop_names_url
                
            except:
                    shop_dict["url"] = "Pending"
            
            shops.append(shop_dict)

        next_button = driver.find_element_by_partial_link_text('Next')
        next_button.click()
except:
    pass
shops

[{'name': '1. VAPE IT STORE II',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=pPfKAhNUVco%3D'},
 {'name': '2. VAPE IT STORE I',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=KVyHlaQ1Wxs%3D'},
 {'name': '3. VAPEPAD THE',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=xCkmCTJY5kU%3D'},
 {'name': '4. VAPE FROG', 'url': 'Pending'},
 {'name': '5. VAPE FROG', 'url': 'Pending'},
 {'name': '6. VAPE LOFT THE',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=gBYJlLwaIhM%3D'},
 {'name': '7. VAPE N CIGAR',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=mJryc8kHJic%3D'},
 {'name': '8. VAPE DOJO',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=bUoW2ojV%2FXY%3D'},
 {'name': '9. VAPE HAVEN', 'url': 'Pending'},
 {'name': '10. VAPORS LOUNGE', 'url': 'Pending'},
 {'name': '11. VAPORFI SOUTHSIDE',
  'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.js

### Save this data as a csv

The filename should be `vape-shops-basic.csv`.

In [19]:
new_df = pd.DataFrame(shops)
new_df.head()

,name,url
0,1. VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicense...
1,2. VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicense...
2,3. VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicense...
3,4. VAPE FROG,Pending
4,5. VAPE FROG,Pending


In [20]:
new_df.to_csv("vape-shops-basic.csv", index=False)

# Okay, let's scrape!

All right, get the actual data!

### Look at the URL of your first row

- *TIP: Remember `pd.set_option('display.max_colwidth', -1)` will let you see alllll of your strings*

In [21]:
pd.set_option('display.max_colwidth', -1)
new_df.head(1)

,name,url
0,1. VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=pPfKAhNUVco%3D


### Use Selenium to visit that page

In [22]:
url = new_df['url'][0]
driver.get(url)

### Now, just like you did before, grab the additional data

You should probably save it into a dictionary! Don't try to put it into the dataframe yet, though. You want:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

.

- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*
- *TIP: If you've gotten addicted to xpath, total amount paid and issued by might not work with it when doing other shops. You'll want to test it!*

In [23]:
url = new_df['url'][0]
driver.get(url)

shop_names = driver.find_elements_by_class_name("searchfieldtitle")

addresses = driver.find_elements_by_tag_name("table")[3]
mailing_add = addresses.find_elements_by_tag_name('tr')[2].find_elements_by_tag_name('td')[0].text
mailing_add_clean = mailing_add.replace("\n", "")
mailing_add_clean

location_add = addresses.find_elements_by_tag_name('tr')[4].find_elements_by_tag_name('td')[0].text
location_add_clean = location_add.replace("\n", "")
location_add_clean

license_info = addresses.find_elements_by_tag_name('tr')[2].find_elements_by_tag_name('td')[1].text
license_info_clean = license_info.replace("\n", "")
license_info_clean

amount_paid = driver.find_elements_by_tag_name("table")[4]
total_amount = amount_paid.find_elements_by_tag_name('tr')[7].find_elements_by_tag_name('td')[1].text
total_amount

issued = driver.find_elements_by_tag_name("table")[5]
issued_by = issued.find_elements_by_tag_name('tr')[1].text
issued_by_clean = issued_by.replace("\n", "")
issued_by_clean

shop_dict = {
    
    "mailing_address": mailing_add_clean,
    "location_address": location_add_clean,
    "license_info": license_info_clean,
    "amount_paid": total_amount,
    "issued_by": issued_by_clean
    
}

shop_dict


{'mailing_address': 'AMIN NARGIS1104 PLANTERS PLACESALISBURY, MD 21804',
 'location_address': 'VAPE IT STORE II1015 S SALISBURY BLVDSALISBURY, MD 21801',
 'license_info': 'License Status: IssuedLicense No.: 22375606Control No.: 22884439Date of Issue: 4/27/2018Months Paid: 12Exp. Date: 4/30/2019Subdivision: 22 Salisbury',
 'amount_paid': '$ 41.00',
 'issued_by': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURTP.O. BOX 198SALISBURY, MARYLAND 21803-0198'}

### Move all of this into one cell

It should visit the URL, then grab the data and put it into a dictionary.

### Change it into a function

You'll want to have this function accept a `row`, and send back a `pd.Series`. You can just use `pd.Series(your_dictionary)` (but it better have a better name than `your_dictionary`!).

- *TIP: Make sure you `return` something!*
- *TIP: Make sure you change everything to reflect the row's url, not the URL you typed in*

In [46]:
driver = webdriver.Chrome()

driver.get(url)
disclaimer_button = driver.find_element_by_xpath('//*[@id="checkbox"]')
disclaimer_button.click()

enter_button = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
enter_button.click()

driver.get(url)
statewide_dropdown = Select(driver.find_element_by_name('slcJurisdiction'))
statewide_dropdown.select_by_visible_text('Statewide')
vape_shops = driver.find_element_by_id("txtTradeName")
vape_shops.send_keys("VAP%") 
submit_search_button = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]')
submit_search_button.click()

def get_all_shops(row):
    try:
        row_url = row['url']
        driver.get(row_url)
        driver.get(row_url)
        
        shop_dict = {}

        shop_names = driver.find_elements_by_class_name("searchfieldtitle")
    
        addresses = driver.find_elements_by_tag_name("table")[3]
        mailing_add = addresses.find_elements_by_tag_name('tr')[2].find_elements_by_tag_name('td')[0].text
        mailing_add_clean = mailing_add.replace("\n", "")
        shop_dict["mailing_address"] = mailing_add_clean
        
        location_add = addresses.find_elements_by_tag_name('tr')[4].find_elements_by_tag_name('td')[0].text
        location_add_clean = location_add.replace("\n", "")
        shop_dict["location_address"] = location_add_clean
        
        license_info = addresses.find_elements_by_tag_name('tr')[2].find_elements_by_tag_name('td')[1].text
        license_info_clean = license_info.replace("\n", "")
        shop_dict["license_info"] = license_info_clean
        
        amount_paid = driver.find_elements_by_tag_name("table")[4]
        total_amount = amount_paid.find_elements_by_tag_name('tr')[7].find_elements_by_tag_name('td')[1].text
        shop_dict["amount_paid"] = total_amount
        
        issued = driver.find_elements_by_tag_name("table")[5]
        issued_by = issued.find_elements_by_tag_name('tr')[1].text
        issued_by_clean = issued_by.replace("\n", "")
        shop_dict["issued_by"] = issued_by_clean
        
        return pd.Series(shop_dict)
        
    except:
        pass


In [47]:
all_shops_df = new_df.apply(get_all_shops, axis=1).join(new_df)
all_shops_df.head()

AttributeError: 'Series' object has no attribute 'join'

### Use your dataframe and `.apply` to pull all of the data from the vape shops

Once you know it's working, use the whole 

- *TIP: Try using it with `.head(3)` first*
- *TIP: You'll want to use `.apply` with your new function*
- *TIP: Issued By and Total Paid are goign to give you problems if you tried to use xpath! Try checking the classes and think about find_elementSSSSS and working backwards instead of forwards.*
- *TIP: You might need a `try`/`except`*
- *TIP: Make sure you're using `axis=1`*
- *TIP: Use `.join` the big thing with all of the `dfs` - make sure you name them right!*

## Save as `vape-total.csv`

Make sure you don't save the index! Open it up in a text editor or Excel to make sure it's correct.